Initial thoughts: 
 - Many features, feature engineering will be required
 - Lets start with multiple linear regression - check if we have multicollinearity

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from IPython.display import display
pd.options.display.max_columns = None

In [24]:
# Read in the train data
data = pd.read_csv("data/train.csv", index_col = 0)
sale_price = data[['SalePrice']]
house_features = data.drop('SalePrice', axis = 'columns')

In [37]:
# Get an overview of the feature set
house_features.describe()
# 36 numerical variables

MSSubClass  LotFrontage        LotArea  OverallQual  OverallCond  \
count  1460.000000  1201.000000    1460.000000  1460.000000  1460.000000   
mean     56.897260    70.049958   10516.828082     6.099315     5.575342   
std      42.300571    24.284752    9981.264932     1.382997     1.112799   
min      20.000000    21.000000    1300.000000     1.000000     1.000000   
25%      20.000000    59.000000    7553.500000     5.000000     5.000000   
50%      50.000000    69.000000    9478.500000     6.000000     5.000000   
75%      70.000000    80.000000   11601.500000     7.000000     6.000000   
max     190.000000   313.000000  215245.000000    10.000000     9.000000   

         YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1   BsmtFinSF2  \
count  1460.000000   1460.000000  1452.000000  1460.000000  1460.000000   
mean   1971.267808   1984.865753   103.685262   443.639726    46.549315   
std      30.202904     20.645407   181.066207   456.098091   161.319273   
min    1872.000000   1950.000000     0.000000     0.000000     0.000000   
25%    1954.000000   1967.000000     0.000000     0.000000     0.000000   
50%    1973.000000   1994.000000     0.000000   383.500000     0.000000   
75%    2000.000000   2004.000000   166.000000   712.250000     0.000000   
max    2010.000000   2010.000000  1600.000000  5644.000000  1474.000000   

         BsmtUnfSF  TotalBsmtSF     1stFlrSF     2ndFlrSF  LowQualFinSF  \
count  1460.000000  1460.000000  1460.000000  1460.000000   1460.000000   
mean    567.240411  1057.429452  1162.626712   346.992466      5.844521   
std     441.866955   438.705324   386.587738   436.528436     48.623081   
min       0.000000     0.000000   334.000000     0.000000      0.000000   
25%     223.000000   795.750000   882.000000     0.000000      0.000000   
50%     477.500000   991.500000  1087.000000     0.000000      0.000000   
75%     808.000000  1298.250000  1391.250000   728.000000      0.000000   
max    2336.000000  6110.000000  4692.000000  2065.000000    572.000000   

         GrLivArea  BsmtFullBath  BsmtHalfBath     FullBath     HalfBath  \
count  1460.000000   1460.000000   1460.000000  1460.000000  1460.000000   
mean   1515.463699      0.425342      0.057534     1.565068     0.382877   
std     525.480383      0.518911      0.238753     0.550916     0.502885   
min     334.000000      0.000000      0.000000     0.000000     0.000000   
25%    1129.500000      0.000000      0.000000     1.000000     0.000000   
50%    1464.000000      0.000000      0.000000     2.000000     0.000000   
75%    1776.750000      1.000000      0.000000     2.000000     1.000000   
max    5642.000000      3.000000      2.000000     3.000000     2.000000   

       BedroomAbvGr  KitchenAbvGr  TotRmsAbvGrd   Fireplaces  GarageYrBlt  \
count   1460.000000   1460.000000   1460.000000  1460.000000  1379.000000   
mean       2.866438      1.046575      6.517808     0.613014  1978.506164   
std        0.815778      0.220338      1.625393     0.644666    24.689725   
min        0.000000      0.000000      2.000000     0.000000  1900.000000   
25%        2.000000      1.000000      5.000000     0.000000  1961.000000   
50%        3.000000      1.000000      6.000000     1.000000  1980.000000   
75%        3.000000      1.000000      7.000000     1.000000  2002.000000   
max        8.000000      3.000000     14.000000     3.000000  2010.000000   

        GarageCars   GarageArea   WoodDeckSF  OpenPorchSF  EnclosedPorch  \
count  1460.000000  1460.000000  1460.000000  1460.000000    1460.000000   
mean      1.767123   472.980137    94.244521    46.660274      21.954110   
std       0.747315   213.804841   125.338794    66.256028      61.119149   
min       0.000000     0.000000     0.000000     0.000000       0.000000   
25%       1.000000   334.500000     0.000000     0.000000       0.000000   
50%       2.000000   480.000000     0.000000    25.000000       0.000000   
75%       2.000000   576.000000   168.000000    68.000000   

In [48]:
house_features.columns[house_features.isnull().any()]
# 19 features with missing data

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [52]:
house_features.loc[:, ['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature']]

LotFrontage Alley MasVnrType  MasVnrArea BsmtQual BsmtCond BsmtExposure  \
Id                                                                              
1            65.0   NaN    BrkFace       196.0       Gd       TA           No   
2            80.0   NaN       None         0.0       Gd       TA           Gd   
3            68.0   NaN    BrkFace       162.0       Gd       TA           Mn   
4            60.0   NaN       None         0.0       TA       Gd           No   
5            84.0   NaN    BrkFace       350.0       Gd       TA           Av   
6            85.0   NaN       None         0.0       Gd       TA           No   
7            75.0   NaN      Stone       186.0       Ex       TA           Av   
8             NaN   NaN      Stone       240.0       Gd       TA           Mn   
9            51.0   NaN       None         0.0       TA       TA           No   
10           50.0   NaN       None         0.0       TA       TA           No   
11           70.0   NaN       None         0.0       TA       TA           No   
12           85.0   NaN      Stone       286.0       Ex       TA           No   
13            NaN   NaN       None         0.0       TA       TA           No   
14           91.0   NaN      Stone       306.0       Gd       TA           Av   
15            NaN   NaN    BrkFace       212.0       TA       TA           No   
16           51.0   NaN       None         0.0       TA       TA           No   
17            NaN   NaN    BrkFace       180.0       TA       TA           No   
18           72.0   NaN       None         0.0      NaN      NaN          NaN   
19           66.0   NaN       None         0.0       TA       TA           No   
20           70.0   NaN       None         0.0       TA       TA           No   
21          101.0   NaN    BrkFace       380.0       Ex       TA           Av   
22           57.0  Grvl       None         0.0       TA       TA           No   
23           75.0   NaN    BrkFace       281.0       Gd       TA           No   
24           44.0   NaN       None         0.0       Gd       TA           No   
25            NaN   NaN       None         0.0       TA       TA           Mn   
26          110.0   NaN      Stone       640.0       Gd       TA           No   
27           60.0   NaN       None         0.0       TA       TA           Mn   
28           98.0   NaN      Stone       200.0       Ex       TA           No   
29           47.0   NaN       None         0.0       TA       TA           Gd   
30           60.0   NaN       None         0.0       TA       TA           No   
...           ...   ...        ...         ...      ...      ...          ...   
1431         60.0   NaN       None         0.0       Gd       Gd           Av   
1432          NaN   NaN       None         0.0       Gd       TA           No   
1433         60.0  Grvl       None         0.0       TA       TA           No   
1434         93.0   NaN    BrkFace       318.0       Gd       TA           No   
1435         80.0   NaN       None         0.0       TA       TA           No   
1436         80.0   NaN    BrkFace       237.0       TA       TA           No   
1437         60.0   NaN       None         0.0       TA       TA           No   
1438         96.0   NaN      Stone       426.0       Ex       TA           Av   
1439         90.0   NaN       None         0.0       TA       TA           No   
1440         80.0   NaN    BrkFace        96.0       TA       TA           No   
1441         79.0   NaN       None         0.0       Ex       TA           No   
1442          NaN   NaN    BrkFace       147.0       Gd       TA           Av   
1443         85.0   NaN      Stone       160.0       Ex       TA           Av   
1444          NaN   NaN       None         0.0       TA       TA           No   
1445         63.0   NaN    BrkFace       106.0       Gd       TA           Av   
1446         70.0   NaN       None         0.0       TA       TA           Gd   
1447          NaN   NaN    BrkFace   

In [58]:
# See if any of the features have zero variance, select only the numeric ones first
import sklearn.feature_selection as fs
hf_numeric = house_features.select_dtypes(include=[np.number])
hf_numeric.apply(np.var)

MSSubClass       1.788113e+03
LotFrontage      5.892581e+02
LotArea          9.955741e+07
OverallQual      1.911369e+00
OverallCond      1.237474e+00
YearBuilt        9.115906e+02
YearRemodAdd     4.259409e+02
MasVnrArea       3.276239e+04
BsmtFinSF1       2.078830e+05
BsmtFinSF2       2.600608e+04
BsmtUnfSF        1.951127e+05
TotalBsmtSF      1.923305e+05
1stFlrSF         1.493477e+05
2ndFlrSF         1.904266e+05
LowQualFinSF     2.362585e+03
GrLivArea        2.759405e+05
BsmtFullBath     2.690838e-01
BsmtHalfBath     5.696378e-02
FullBath         3.033003e-01
HalfBath         2.527205e-01
BedroomAbvGr     6.650380e-01
KitchenAbvGr     4.851567e-02
TotRmsAbvGrd     2.640094e+00
Fireplaces       4.153101e-01
GarageYrBlt      6.091405e+02
GarageCars       5.580972e-01
GarageArea       4.568120e+04
WoodDeckSF       1.569905e+04
OpenPorchSF      4.386854e+03
EnclosedPorch    3.732992e+03
3SsnPorch        8.589172e+02
ScreenPorch      3.106760e+03
PoolArea         1.613110e+03
MiscVal   

In [59]:
hf_numeric

MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
Id                                                                            
1             60         65.0     8450            7            5       2003   
2             20         80.0     9600            6            8       1976   
3             60         68.0    11250            7            5       2001   
4             70         60.0     9550            7            5       1915   
5             60         84.0    14260            8            5       2000   
6             50         85.0    14115            5            5       1993   
7             20         75.0    10084            8            5       2004   
8             60          NaN    10382            7            6       1973   
9             50         51.0     6120            7            5       1931   
10           190         50.0     7420            5            6       1939   
11            20         70.0    11200            5            5       1965   
12            60         85.0    11924            9            5       2005   
13            20          NaN    12968            5            6       1962   
14            20         91.0    10652            7            5       2006   
15            20          NaN    10920            6            5       1960   
16            45         51.0     6120            7            8       1929   
17            20          NaN    11241            6            7       1970   
18            90         72.0    10791            4            5       1967   
19            20         66.0    13695            5            5       2004   
20            20         70.0     7560            5            6       1958   
21            60        101.0    14215            8            5       2005   
22            45         57.0     7449            7            7       1930   
23            20         75.0     9742            8            5       2002   
24           120         44.0     4224            5            7       1976   
25            20          NaN     8246            5            8       1968   
26            20        110.0    14230            8            5       2007   
27            20         60.0     7200            5            7       1951   
28            20         98.0    11478            8            5       2007   
29            20         47.0    16321            5            6       1957   
30            30         60.0     6324            4            6       1927   
...          ...          ...      ...          ...          ...        ...   
1431          60         60.0    21930            5            5       2005   
1432         120          NaN     4928            6            6       1976   
1433          30         60.0    10800            4            6       1927   
1434          60         93.0    10261            6            5       2000   
1435          20         80.0    17400            5            5       1977   
1436          20         80.0     8400            6            9       1962   
1437          20         60.0     9000            4            6       1971   
1438          20         96.0    12444            8            5       2008   
1439          20         90.0     7407            6            7       1957   
1440          60         80.0    11584            7            6       1979   
1441          70         79.0    11526            6            7       1922   
1442         120          NaN     4426            6            5       2004   
1443          60         85.0    11003           10            5       2008   
1444          30          NaN     8854            6            6       1916   
1445          20         63.0     8500            7            5       2004   
1446          85         70.0     8400            6            5       1966   
1447          20          NaN    26142            5            7       1962   
1448          60         80.0    10000            8     